## Banco de dados DATASET11 - Bearing Classification

### Importação de Bibliotecas

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.signal import iirnotch
import scipy.io


### Carregamento de arquivos

In [54]:

letters = ['H', 'I', 'O'] # 0-healthy, I-inner fault, O-outer fault
speeds = ['A', 'B', 'C', 'D']
trials = ['1','2', '3']
state = 0
dataset = []
for letter in letters:
    for speed in speeds:
        for trial in trials:
            path_signal = f'../../Dados/Rotor_Bearing/Dataset11/Data/{letter}-{speed}-{trial}.mat'
            df_signal = scipy.io.loadmat(path_signal)
            signal = []
            for i in range(0,len(df_signal['Channel_1'])):
                sample = [df_signal['Channel_1'][i][0], df_signal['Channel_2'][i][0]]
                signal.append(sample)
            signal.append(state)
        dataset.append(signal)
    state +=1

In [ ]:
## parei aqui
for i in dataset:
    print(i)

In [59]:
import numpy as np

# Example boolean array (True for correct, False for incorrect)
correctness_array = np.array([True, True, False, True, False, True])

# Calculate accuracy as a percentage
accuracy_percentage = np.mean(correctness_array) * 100

# Print the result
print(f'Accuracy: {accuracy_percentage:.2f}%')

##### Visualização dos sinais

In [35]:
import matplotlib.pyplot as plt
import numpy as np

# Example dataset (replace this with your actual dataset)

# Iterate through each element in the dataset
for element in dataset:
    # Extract 'Channel_1' and 'Channel_2' arrays
    channel_1_data = element[0]
    channel_2_data = element[1]

    # Create a new figure for each element
    plt.figure()

    # Plot the data
    plt.plot(channel_2_data, channel_1_data, marker='o', linestyle='-', label=f'Element')

    # Customize plot (add title, labels, legend, etc. as needed)
    # plt.title(f'Element {idx + 1} Plot')
    plt.xlabel('Channel_2')
    plt.ylabel('Channel_1')
    plt.legend()

    # Show or save the figure
    # plt.show()  # Uncomment this line to display figures interactively
    # plt.savefig(f'element_{idx + 1}_plot.png')  # Save figures to files

# Show the plots (if not saving to files)
plt.show()


TypeError: list indices must be integers or slices, not tuple

### Tratamento dos dados em vetores de entrada

### Aplicação de técnicas de processamento de sinais - Cálculos estatísticos

### Aplicação de técnicas de inteligência artificial

In [ ]:
## Kohonen Map (clusterizacao)

## MLP (classificacao)

## LVQ (rotulacao)

### Gráficos - Apresentação dos resultados

#### CHATGPT HELP
